In [1]:
import pandas as pd 
import numpy as np
import re

hashtag_rex = re.compile("(#[A-Za-z0-9]+)")
spaces_rex = re.compile("\s\s+")

In [2]:
file = 'COVID-post_enero-julio_TWR-DEF_final.csv'
df = pd.read_csv(file,sep=';')
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4
0,"Aug 1, 2020 @ 15:32:45.000",Spain,Huffpost ES,Online,"Madrid, la gran ciudad europea con mayor exces...",NaN,NaN,NaN,True,False,False,False
1,"Aug 1, 2020 @ 14:04:00.000",Spain,Huffpost ES,Online,"Rifirrafe entre Fauci, el 'Fernando Simón de E...",NaN,Fernando Simón,Political and Partisian Source,True,False,True,True
2,"Aug 1, 2020 @ 12:43:39.000",Spain,Huffpost ES,Online,El esperanzador mensaje de una investigadora e...,NaN,NaN,NaN,True,False,False,False
3,"Aug 1, 2020 @ 11:34:04.000",Germany,BILD,Print,Coronavirus-News im Live-Ticker - Demonstrante...,NaN,NaN,NaN,True,True,True,True
4,"Aug 1, 2020 @ 10:54:34.000",Germany,BILD,Print,„Keine Touristen in Quarantäne-Hotel“ - Hotelb...,NaN,NaN,NaN,True,False,True,True


In [3]:
def find_verbs_actors(message,verbs,actors,rule):
    if actors == '' and rule:
        return 'Actors_not_identified'

    if verbs == '' or actors == '' or message == '':
        return ''
    
    verbs_splitted = verbs.lower().split(",")
    actors_splitted = actors.lower().split(",")
    message_splitted = message.replace(".","").replace(",","").replace("’"," ").lower().split(" ")
    
    idx_verbs = []
    idx_actors = []
    
    for i in range(len(verbs_splitted)):
        v_aux = verbs_splitted[i].split(' ')[0]
        for j in range(len(message_splitted)):
            if v_aux == message_splitted[j]:
                idx_verbs.append(j)
                #break
                
    for i in range(len(actors_splitted)):
        a_aux = actors_splitted[i].split(' ')[-1]
        for j in range(len(message_splitted)):
            if a_aux == message_splitted[j]:
                idx_actors.append(j)
                #break
    idx_verbs = np.array(idx_verbs)
    idx_actors = np.array(idx_actors)
    
    if len(idx_actors) == 0 and rule:
        return 'Actors_not_found'
    if len(idx_actors) == 0:
        return ''
    
    pairs = []
    
    for idx_verb in idx_verbs:
        
        act = message_splitted[idx_actors[np.argmin(np.abs(idx_actors - idx_verb))]]
        vrb = message_splitted[idx_verb]
        pairs.append((act,vrb))
    return str(pairs)

In [4]:
filtered_df = df[(~df['Verbs R2'].isna()) & (~df['Actors R1'].isna()) ]
filtered_df.head(10)

,Date,Country,MediaOutlet,MediaOrigin,Message,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4
28,"Aug 1, 2020 @ 02:34:00.000",Chile,Canal 13,TV,🔶 Cuenta Pública: Piñera anuncia plan económic...,anuncia,piñera,Political and Partisian Source,True,False,False,False
47,"Aug 1, 2020 @ 01:32:14.000",Chile,Cooperativa,Radio,[Video] Piñera reconoció que ayuda por pandemi...,Reconoció,piñera,Political and Partisian Source,True,False,False,False
49,"Aug 1, 2020 @ 00:59:56.000",Chile,TVN,TV,El Mandatario describió seis medidas para enfr...,describió,Mandatario,Political and Partisian Source,True,False,False,False
52,"Aug 1, 2020 @ 00:32:28.000",Chile,Canal 13,TV,Piñera señala que aportes del Estado para enfr...,señala,piñera,Political and Partisian Source,True,True,False,True
61,"Aug 1, 2020 @ 00:13:27.000",Chile,Biobio.cl,Radio,🔴AHORA | #CuentaPública Presidente @sebastianp...,asegura,presidente,Political and Partisian Source,True,False,False,True
65,"Aug 1, 2020 @ 00:08:42.000",United Kingdom,The Guardian,Print,Coronavirus live news: WHO reports record glob...,reports,England,Political and Partisian Source,True,False,False,False
70,"Jul 31, 2020 @ 23:53:00.000",United States,CNN USA,TV,MLB Commissioner Rob Manfred warned union lead...,"warned,call,reports",Rob Manfred,Sport Source,True,False,False,True
75,"Jul 31, 2020 @ 23:45:00.000",United States,CNN USA,TV,A coronavirus outbreak at a Georgia sleep-away...,"according to,says",Centers Disease,Health Source,True,False,False,False
120,"Jul 31, 2020 @ 22:38:00.000",United States,CNN USA,TV,The pandemic poses a challenge after years of ...,speaks,"Twitter,Facebook","Business Source,Business Source",True,False,False,False
123,"Jul 31, 2020 @ 22:30:00.000",Chile,Elmostrador.cl,Online,Patente de alcoholes: diputados piden que muni...,piden,diputados,Political and Partisian Source,True,False,False,False


In [5]:
x = filtered_df.iloc[5]
print(x)
mess, verb, act, rule = x['Message'], x['Verbs R2'], x['Actors R1'], (x['rule 3'] or x['rule 2'])

find_verbs_actors(mess, verb, act, rule)

Date                                       Aug 1, 2020 @ 00:08:42.000
Country                                                United Kingdom
MediaOutlet                                              The Guardian
MediaOrigin                                                     Print
Message             Coronavirus live news: WHO reports record glob...
Verbs R2                                                      reports
Actors R1                                                     England
Actors R1 - Type                       Political and Partisian Source
rule 1                                                           True
rule 2                                                          False
rule 3                                                          False
rule 4                                                          False
Name: 65, dtype: object


"[('england', 'reports')]"

In [6]:
print(len(df))

314768


In [7]:
pairs = []
df = df.fillna('')
for i in range(len(df)):
    if i % 25000 == 0:
        print(i)
    sub_df = df.loc[i,['Message','Verbs R2','Actors R1', 'rule 3', 'rule 4']]
    mess, verb, act, rule = sub_df['Message'], sub_df['Verbs R2'], sub_df['Actors R1'], (sub_df['rule 3'])
    
    p = find_verbs_actors(mess, verb, act, rule)
    pairs.append(p)

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000


In [8]:
df['Actor_verbs'] = pd.Series(pairs)
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4,Actor_verbs
0,"Aug 1, 2020 @ 15:32:45.000",Spain,Huffpost ES,Online,"Madrid, la gran ciudad europea con mayor exces...",,,,True,False,False,False,
1,"Aug 1, 2020 @ 14:04:00.000",Spain,Huffpost ES,Online,"Rifirrafe entre Fauci, el 'Fernando Simón de E...",,Fernando Simón,Political and Partisian Source,True,False,True,True,
2,"Aug 1, 2020 @ 12:43:39.000",Spain,Huffpost ES,Online,El esperanzador mensaje de una investigadora e...,,,,True,False,False,False,
3,"Aug 1, 2020 @ 11:34:04.000",Germany,BILD,Print,Coronavirus-News im Live-Ticker - Demonstrante...,,,,True,True,True,True,Actors_not_identified
4,"Aug 1, 2020 @ 10:54:34.000",Germany,BILD,Print,„Keine Touristen in Quarantäne-Hotel“ - Hotelb...,,,,True,False,True,True,Actors_not_identified


In [9]:
mask = df['Actor_verbs'] == 'Actors_not_identified'
df.loc[mask,'Actors R1'] = df[mask]['Actor_verbs']
df.loc[mask,'Actors R1 - Type'] = df[mask]['Actor_verbs']

In [10]:
df.to_csv(file.replace("final","final1"),sep=";",index=False,encoding='utf-8-sig')

In [11]:
file = 'COVID-post_enero-julio_IGRAM-DEF_final.csv'
df = pd.read_csv(file,sep=';')
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,ImageText,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4
0,"Jan 2, 2020 @ 17:36:00.000",United States,Los Angeles Times,Print,The Trump administration announced a temporary...,NaN,announced,"White House,Trump","Political and Partisian Source,Political and P...",True,False,True,True
1,"Jan 8, 2020 @ 12:15:00.000",United States,Los Angeles Times,Print,"An estimated 46,000 Haitians reside in the U.S...",NaN,estimated,NaN,NaN,True,False,False,False
2,"Jan 12, 2020 @ 08:44:00.000",United Kingdom,Sky News,TV,Knife crime offences are at a record high but ...,NaN,"said,spoke",NaN,NaN,False,False,True,True
3,"Jan 13, 2020 @ 14:02:37.000",Chile,Mega,TV,La Organización Mundial de la Salud #OMS y las...,Nuevo virus logra salir de China y se registra...,"Según el,anunciaron","autoridades,Wuhan,familia,Organización Mundial...","Political and Partisian Source,Political and P...",False,False,False,False
4,"Jan 13, 2020 @ 19:52:08.000",Chile,Canal 13,TV,Una de las expositoras más esperadas del #Cong...,"CONGRESO T13 ""Está documentado que los hombres...",expuso,Escritora,Performer-Artist or Celebrity Source,False,False,True,True


In [12]:
pairs = []
df = df.fillna('')
for i in range(len(df)):
    if i % 25000 == 0:
        print(i)
    sub_df = df.loc[i,['Message','Verbs R2','Actors R1', 'rule 3', 'rule 4']]
    mess, verb, act, rule = sub_df['Message'], sub_df['Verbs R2'], sub_df['Actors R1'], (sub_df['rule 3'])
    
    p = find_verbs_actors(mess, verb, act, rule)
    pairs.append(p)

0
25000


In [13]:
df['Actor_verbs'] = pd.Series(pairs)
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,ImageText,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4,Actor_verbs
0,"Jan 2, 2020 @ 17:36:00.000",United States,Los Angeles Times,Print,The Trump administration announced a temporary...,,announced,"White House,Trump","Political and Partisian Source,Political and P...",True,False,True,True,"[('trump', 'announced')]"
1,"Jan 8, 2020 @ 12:15:00.000",United States,Los Angeles Times,Print,"An estimated 46,000 Haitians reside in the U.S...",,estimated,,,True,False,False,False,
2,"Jan 12, 2020 @ 08:44:00.000",United Kingdom,Sky News,TV,Knife crime offences are at a record high but ...,,"said,spoke",,,False,False,True,True,Actors_not_identified
3,"Jan 13, 2020 @ 14:02:37.000",Chile,Mega,TV,La Organización Mundial de la Salud #OMS y las...,Nuevo virus logra salir de China y se registra...,"Según el,anunciaron","autoridades,Wuhan,familia,Organización Mundial...","Political and Partisian Source,Political and P...",False,False,False,False,"[('wuhan', 'según'), ('autoridades', 'anunciar..."
4,"Jan 13, 2020 @ 19:52:08.000",Chile,Canal 13,TV,Una de las expositoras más esperadas del #Cong...,"CONGRESO T13 ""Está documentado que los hombres...",expuso,Escritora,Performer-Artist or Celebrity Source,False,False,True,True,"[('escritora', 'expuso')]"


In [14]:
mask = df['Actor_verbs'] == 'Actors_not_identified'
df.loc[mask,'Actors R1'] = df[mask]['Actor_verbs']
df.loc[mask,'Actors R1 - Type'] = df[mask]['Actor_verbs']

In [15]:
df.to_csv(file.replace("final","final1"),sep=";",index=False,encoding='utf-8-sig')

In [16]:
file = 'COVID-post_enero-julio_FBK-DEF_final.csv'
df = pd.read_csv(file,sep=';')
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,LinkText,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4
0,"Jan 2, 2020 @ 19:19:38.000",United States,Los Angeles Times,Print,The Trump administration has announced a tempo...,"Trump administration imposed ban on some, but ...",announced,Trump,Political and Partisian Source,False,False,False,True
1,"Jan 2, 2020 @ 22:39:39.000",United States,CBS US,Radio,"The U.S. faces an epidemic of cyberattacks, wi...",Ransomware attack shuts down some Michigan sch...,NaN,NaN,NaN,False,False,False,False
2,"Jan 3, 2020 @ 01:40:00.000",United States,Los Angeles Times,Print,California has expanded a quarantine zone in S...,California expands local quarantine to halt ci...,state,NaN,NaN,False,False,False,False
3,"Jan 3, 2020 @ 08:16:13.000",Spain,24hTVE,TV,"La decisión, que entrará en vigor en 30 días, ...",Prohíben en Estados Unidos los cartuchos para ...,NaN,"Donald Trump,Trump","Political and Partisian Source,Political and P...",False,False,True,True
4,"Jan 3, 2020 @ 14:40:00.000",Chile,Elmostrador.cl,Online,CULTURA| La escasez hídrica producto del cambi...,Académica chilena en Nature: Más inversión en ...,NaN,NaN,NaN,False,False,False,True


In [17]:
pairs = []
df = df.fillna('')
for i in range(len(df)):
    if i % 25000 == 0:
        print(i)
    sub_df = df.loc[i,['Message','Verbs R2','Actors R1', 'rule 3', 'rule 4']]
    mess, verb, act, rule = sub_df['Message'], sub_df['Verbs R2'], sub_df['Actors R1'], (sub_df['rule 3'])
    
    p = find_verbs_actors(mess, verb, act, rule)
    pairs.append(p)

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000


In [18]:
df['Actor_verbs'] = pd.Series(pairs)
df.head()

,Date,Country,MediaOutlet,MediaOrigin,Message,LinkText,Verbs R2,Actors R1,Actors R1 - Type,rule 1,rule 2,rule 3,rule 4,Actor_verbs
0,"Jan 2, 2020 @ 19:19:38.000",United States,Los Angeles Times,Print,The Trump administration has announced a tempo...,"Trump administration imposed ban on some, but ...",announced,Trump,Political and Partisian Source,False,False,False,True,"[('trump', 'announced')]"
1,"Jan 2, 2020 @ 22:39:39.000",United States,CBS US,Radio,"The U.S. faces an epidemic of cyberattacks, wi...",Ransomware attack shuts down some Michigan sch...,,,,False,False,False,False,
2,"Jan 3, 2020 @ 01:40:00.000",United States,Los Angeles Times,Print,California has expanded a quarantine zone in S...,California expands local quarantine to halt ci...,state,,,False,False,False,False,
3,"Jan 3, 2020 @ 08:16:13.000",Spain,24hTVE,TV,"La decisión, que entrará en vigor en 30 días, ...",Prohíben en Estados Unidos los cartuchos para ...,,"Donald Trump,Trump","Political and Partisian Source,Political and P...",False,False,True,True,
4,"Jan 3, 2020 @ 14:40:00.000",Chile,Elmostrador.cl,Online,CULTURA| La escasez hídrica producto del cambi...,Académica chilena en Nature: Más inversión en ...,,,,False,False,False,True,


In [19]:
mask = df['Actor_verbs'] == 'Actors_not_identified'
df.loc[mask,'Actors R1'] = df[mask]['Actor_verbs']
df.loc[mask,'Actors R1 - Type'] = df[mask]['Actor_verbs']

In [20]:
df.to_csv(file.replace("final","final1"),sep=";",index=False,encoding='utf-8-sig')

In [21]:
asdfg = hashtag_rex.sub("",mess)
spaces_rex.sub(" ",asdfg)

'Muchos las están desempolvando para evitar posibles contagios.'